# Advanced Assignment: E-Commerce System

This notebook contains a comprehensive implementation of an e-commerce system, demonstrating advanced OOP concepts in Python.

In [1]:
from abc import ABC, abstractmethod
import datetime

class Product:
    def __init__(self, product_id, name, price, inventory):
        self._product_id = product_id
        self._name = name
        self._price = price
        self._inventory = inventory

    @property
    def product_id(self):
        return self._product_id

    @property
    def name(self):
        return self._name

    @property
    def price(self):
        return self._price

    @price.setter
    def price(self, value):
        if value < 0:
            raise ValueError("Price cannot be negative")
        self._price = value

    @property
    def inventory(self):
        return self._inventory

    @inventory.setter
    def inventory(self, value):
        if value < 0:
            raise ValueError("Inventory cannot be negative")
        self._inventory = value

    def __str__(self):
        return f"{self.name} (ID: {self.product_id}) - ${self.price:.2f} (In stock: {self.inventory})"

class User(ABC):
    def __init__(self, user_id, name, email):
        self._user_id = user_id
        self._name = name
        self._email = email

    @property
    def user_id(self):
        return self._user_id

    @property
    def name(self):
        return self._name

    @property
    def email(self):
        return self._email

    @abstractmethod
    def display_info(self):
        pass

class Customer(User):
    def __init__(self, user_id, name, email):
        super().__init__(user_id, name, email)
        self.shopping_cart = ShoppingCart()
        self.order_history = []

    def display_info(self):
        return f"Customer: {self.name} (ID: {self.user_id}, Email: {self.email})"

    def place_order(self):
        if not self.shopping_cart.items:
            return "Shopping cart is empty"
        order = Order(self, self.shopping_cart.items)
        self.order_history.append(order)
        self.shopping_cart.clear()
        return order

class Admin(User):
    def __init__(self, user_id, name, email, role):
        super().__init__(user_id, name, email)
        self._role = role

    @property
    def role(self):
        return self._role

    def display_info(self):
        return f"Admin: {self.name} (ID: {self.user_id}, Email: {self.email}, Role: {self.role})"

class ShoppingCart:
    def __init__(self):
        self.items = {}

    def add_item(self, product, quantity):
        if quantity <= 0:
            raise ValueError("Quantity must be positive")
        if product.inventory < quantity:
            raise ValueError("Not enough inventory")
        if product in self.items:
            self.items[product] += quantity
        else:
            self.items[product] = quantity

    def remove_item(self, product, quantity):
        if product not in self.items:
            raise ValueError("Product not in cart")
        if quantity <= 0:
            raise ValueError("Quantity must be positive")
        if self.items[product] < quantity:
            raise ValueError("Not enough items in cart")
        self.items[product] -= quantity
        if self.items[product] == 0:
            del self.items[product]

    def get_total(self):
        return sum(product.price * quantity for product, quantity in self.items.items())

    def clear(self):
        self.items.clear()

    def __str__(self):
        return "\n".join([f"{product.name}: {quantity}" for product, quantity in self.items.items()])

class Order:
    def __init__(self, customer, items):
        self.order_id = self._generate_order_id()
        self.customer = customer
        self.items = dict(items)  # Create a copy of the items
        self.total = sum(product.price * quantity for product, quantity in self.items.items())
        self.status = "Pending"
        self.order_date = datetime.datetime.now()

    def _generate_order_id(self):
        return f"ORD-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

    def process_order(self):
        for product, quantity in self.items.items():
            if product.inventory < quantity:
                raise ValueError(f"Not enough inventory for {product.name}")
            product.inventory -= quantity
        self.status = "Processed"
        return f"Order {self.order_id} for {self.customer.name} has been processed. Total: ${self.total:.2f}"

    def __str__(self):
        return f"Order {self.order_id} - Status: {self.status}, Total: ${self.total:.2f}"

class PaymentProcessor(ABC):
    @abstractmethod
    def process_payment(self, amount):
        pass

class CreditCardPayment(PaymentProcessor):
    def __init__(self, card_number, expiry_date, cvv):
        self.card_number = card_number
        self.expiry_date = expiry_date
        self.cvv = cvv

    def process_payment(self, amount):
        # Yoooop !!! In a real system, this would interact with a payment gateway
        return f"Processed credit card payment of ${amount:.2f}"

class PayPalPayment(PaymentProcessor):
    def __init__(self, email):
        self.email = email

    def process_payment(self, amount):
        # Yoooop !!! In a real system, this would interact with PayPal's API
        return f"Processed PayPal payment of ${amount:.2f}"

class InventoryManager:
    @staticmethod
    def update_inventory(product, quantity):
        if quantity > product.inventory:
            raise ValueError(f"Not enough inventory for {product.name}")
        product.inventory -= quantity
        return f"Updated inventory for {product.name}. New stock: {product.inventory}"

    @staticmethod
    def restock_product(product, quantity):
        if quantity <= 0:
            raise ValueError("Restock quantity must be positive")
        product.inventory += quantity
        return f"Restocked {product.name}. New stock: {product.inventory}"

class ECommerceSystem:
    def __init__(self):
        self.products = {}
        self.users = {}
        self.orders = []

    def add_product(self, product):
        self.products[product.product_id] = product

    def remove_product(self, product_id):
        if product_id not in self.products:
            raise ValueError("Product not found")
        del self.products[product_id]

    def add_user(self, user):
        self.users[user.user_id] = user

    def remove_user(self, user_id):
        if user_id not in self.users:
            raise ValueError("User not found")
        del self.users[user_id]

    def process_order(self, order):
        order.process_order()
        self.orders.append(order)

    def get_order_history(self, customer):
        return customer.order_history

    def generate_inventory_report(self):
        return "\n".join([str(product) for product in self.products.values()])

# Test the E-commerce System
ecommerce_system = ECommerceSystem()

# Add products
laptop = Product("P001", "Laptop", 999.99, 10)
smartphone = Product("P002", "Smartphone", 499.99, 20)
ecommerce_system.add_product(laptop)
ecommerce_system.add_product(smartphone)

# Add users
customer = Customer("C001", "Eze Ikenna", "ezeik@gmail.com")
admin = Admin("A001", "Emmanuel Ugochukwu", "eugochukwu77@gmail.com", "Inventory Manager")
ecommerce_system.add_user(customer)
ecommerce_system.add_user(admin)

print(customer.display_info())
print(admin.display_info())

# Customer adds items to cart
customer.shopping_cart.add_item(laptop, 1)
customer.shopping_cart.add_item(smartphone, 2)

print(f"\nShopping Cart:")
print(customer.shopping_cart)
print(f"Shopping Cart Total: ${customer.shopping_cart.get_total():.2f}")

# Place order
order = customer.place_order()
ecommerce_system.process_order(order)
print(f"\n{order}")

# Process payment
payment_processor = CreditCardPayment("1234-5678-9012-3456", "12/25", "123")
print(payment_processor.process_payment(order.total))

# Check inventory after order
print(f"\nUpdated Inventory:")
print(ecommerce_system.generate_inventory_report())

# Admin restocks a product
print(f"\nRestocking:")
print(InventoryManager.restock_product(laptop, 5))

# Final inventory report
print(f"\nFinal Inventory Report:")
print(ecommerce_system.generate_inventory_report())

# Get order history for customer
print(f"\nOrder History for {customer.name}:")
for order in ecommerce_system.get_order_history(customer):
    print(order)

Customer: John Doe (ID: C001, Email: john@example.com)
Admin: Jane Smith (ID: A001, Email: jane@example.com, Role: Inventory Manager)

Shopping Cart:
Laptop: 1
Smartphone: 2
Shopping Cart Total: $1999.97

Order ORD-20250116215749 - Status: Processed, Total: $1999.97
Processed credit card payment of $1999.97

Updated Inventory:
Laptop (ID: P001) - $999.99 (In stock: 9)
Smartphone (ID: P002) - $499.99 (In stock: 18)

Restocking:
Restocked Laptop. New stock: 14

Final Inventory Report:
Laptop (ID: P001) - $999.99 (In stock: 14)
Smartphone (ID: P002) - $499.99 (In stock: 18)

Order History for John Doe:
Order ORD-20250116215749 - Status: Processed, Total: $1999.97
